# 如何绕过反扒机制

为了避开常见的反爬机制，需要3个措施：

1. 使用代理IP，在请求时进行轮换
2. 使用不同的User-Agent
3. 降低请求频率，随机化前后两次的请求时间

教程：

* https://www.scrapehero.com/how-to-prevent-getting-blacklisted-while-scraping/
* https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/
* https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

In [7]:
import os
from typing import List
import re

import requests
from bs4 import BeautifulSoup

## 1. 使用代理IP

首先，我们需要一个函数检测代理IP的有效性。

通过访问[httpbin.org](https://httpbin.org/ip)，可以返回发送请求的真实IP，如果该IP与代理相同，证明代理是有效的。

In [2]:
def test_proxy(proxy: str) -> bool:
    """测试代理IP是否有效

    Args:
        proxy(str): 代理IP
            基础格式：'http://ip:port'
            需要认证：'http://user:pass@ip:port/'

    Returns:
        代理有效返回True, 无效返回False
    """
    proxies = {"http": proxy, "https": proxy}
    try:
        resp = requests.get(
            "https://httpbin.org/ip", proxies=proxies, timeout=3)
        resp.raise_for_status()
    except Exception:
        return False
    else:
        origin = resp.json()["origin"]
        proxy_ip = re.search(
            r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", proxy).group()
        return proxy_ip == origin

### 1.1 免费代理

从[free-proxy-list.net](https://free-proxy-list.net/)获取免费的代理IP。

网上虽然有大量的免费代理可以使用，但效果很糟糕，因为使用的人太多。

In [3]:
def get_freeproxy_proxies() -> List[str]:
    resp = requests.get("https://free-proxy-list.net/")
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    proxies = []
    for row in soup.find("table", {"id": "proxylisttable"}).tbody.children:
        proxy = ":".join([col.get_text() for col in row.find_all("td")[:2]])
        proxies.append(proxy)

    return proxies

In [5]:
proxies = get_freeproxy_proxies()
proxies[:5]

['119.28.65.21:80',
 '192.41.19.53:3128',
 '64.137.175.85:3128',
 '208.80.28.208:8080',
 '216.250.236.11:3128']

In [6]:
for proxy in proxies[:5]:
    res = test_proxy(proxy)
    print(f"{proxy}: {res}")

119.28.65.21:80: False
192.41.19.53:3128: True
64.137.175.85:3128: False
208.80.28.208:8080: False
216.250.236.11:3128: True


### 1.2 付费代理

WeShare: 免费用户可获得10个私有代理IP，每月1G流量。

为什么使用WeShare:

* 免费体验计划
* 付费成本较低
* 代理质量比较高
* 提供REST API查询代理IP和使用情况

In [8]:
def get_weshare_proxies() -> List[str]:
    url = os.getenv("WESHARE_URL")
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text.split()

In [12]:
proxies = get_weshare_proxies()
proxies

['209.127.191.180:80',
 '45.130.255.243:80',
 '185.164.56.20:80',
 '45.130.255.198:80',
 '185.30.232.123:80',
 '45.95.96.132:80',
 '45.95.96.237:80',
 '45.95.96.187:80',
 '45.94.47.66:80',
 '193.8.56.119:80']

In [13]:
username = os.getenv("WESHARE_USERNAME")
password = os.getenv("WESHARE_PASSWORD")
for proxy in proxies:
    res = test_proxy(f"http://{username}:{password}@{proxy}/")
    print(f"{proxy}: {res}")

209.127.191.180:80: True
45.130.255.243:80: True
185.164.56.20:80: True
45.130.255.198:80: False
185.30.232.123:80: True
45.95.96.132:80: False
45.95.96.237:80: False
45.95.96.187:80: False
45.94.47.66:80: True
193.8.56.119:80: True


## 2. 轮换User-Agent

#### 什么是User-Agent?

User-Agent即用户代理，这是浏览器/爬虫发送给服务器的一个字符串，描述一些重要信息，例如浏览器版本，操作系统等，服务器可以利用这些信息做优化，例如优化网页加载速度。User-Agent通常作为请求头(request header)的一部分发送给服务器。

#### 为什么要使用User-Agent?

反扒机制一般会对User-Agent进行检查，如果发现不是浏览器发出的请求，很可能会拒绝请求。

以Python requests为例，默认User-Agent包含'python'字样，很容易就被识别为程序发送的请求。

#### 为什么要轮换User-Agent?

如果要对一个网站进行数千次爬取，最好的办法是每次请求都使用不同的IP和headers，伪装成是不同地区不同的浏览器在对网站发出请求，这样反扒机制就很难识别你的爬虫。

#### 如何实现轮换？

1. 半自动轮换。

从网上收集User-Agent，构建一个列表，每次请求时随机抽取，更新headers，然后发送请求。

* https://deviceatlas.com/blog/list-of-user-agent-strings
* https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent
* https://www.whoishostingthis.com/tools/user-agent/

2. 自动轮换。使用三方库'fake-headers'自动生成用户代理。

有时候仅仅提供User-Agent是不够的，有的网站会要求提供特定的请求头字段，在爬取之前先用浏览器开发工具检查请求头，复制下来，然后在请求时使用。

In [14]:
resp = requests.get("http://httpbin.org/headers")
resp.json()

{'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.24.0',
  'X-Amzn-Trace-Id': 'Root=1-5f9ebb9e-24c315134621879c26a93b7c'}}

In [15]:
from pprint import pprint
from fake_headers import Headers

In [17]:
header = Headers(headers=False)  # 生成最简headers,随机用户代理

for _ in range(3):
    pprint(header.generate())

{'Accept': '*/*',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (X11; Linux i686 on x86_64; rv:58.0.2) '
               'Gecko/20100101 Firefox/58.0.2'}
{'Accept': '*/*',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 '
               'Safari/537.36 OPR/55.0.2994.37'}
{'Accept': '*/*',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; Win64; x64; rv:62.0.2) '
               'Gecko/20100101 Firefox/62.0.2'}


In [18]:
header = Headers(headers=True)  # 生成包含大量信息的请求头

for _ in range(3):
    pprint(header.generate())

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'en-US;q=0.5,en;q=0.3',
 'Cache-Control': 'max-age=0',
 'Connection': 'keep-alive',
 'DNT': '1',
 'Referer': 'https://google.com',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_6) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 '
               'Safari/537.36'}
{'Accept': '*/*',
 'Accept-Language': 'en-US;q=0.5,en;q=0.3',
 'Cache-Control': 'max-age=0',
 'Connection': 'keep-alive',
 'Pragma': 'no-cache',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like '
               'Gecko) Chrome/70.0.3538.80 Safari/537.36 OPR/56.0.3051.116'}
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate, br',
 'Connection': 'keep-alive',
 'Pragma': 'no-cache',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) '
               'AppleWebKit/537.